In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tensorflow_decision_forests --upgrade

In [ ]:
!pip install tensorflow==2.5.0

In [ ]:
!pip install -q wurlitzer

# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
import tensorflow_decision_forests as tfdf

# Load train and test dataset

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
train.head()

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
test.head()

In [ ]:
sample_submission = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')
sample_submission.head()

# EDA

In [ ]:
train.info()

In [ ]:
test.info()

*  #### From the above observation we can see that we dont have null values.
*  #### The datatype of  the features are numerical and integer type.

In [ ]:
train.drop('id', axis=1, inplace = True)
test.drop('id', axis=1, inplace = True)

In [ ]:
train.describe().T\
        .style.bar(subset=['mean'], color=px.colors.qualitative.G10[0])\
        .background_gradient(subset=['std'], cmap='Greens')\
        .background_gradient(subset=['50%'], cmap='BuGn')

In [ ]:
train['target'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x = train['target'])

* #### Target data is highly imbalance

In [ ]:
train_copy = train.copy()

In [ ]:
train.drop('target', axis=1, inplace = True)

In [ ]:
data = pd.concat([train,test])
data.head()

In [ ]:
data.tail()

In [ ]:
corr = data.corr()
fig, ax = plt.subplots(figsize=(30,20))
mask = np.triu(data.corr())
sns.heatmap(corr, vmin=-1, vmax=1, center= 0, linewidths=3, linecolor='white', square = True, mask = mask)

#### There is no correlation between features

In [ ]:
n_rows = 15
n_cols = 5
init = 1
features = data.columns.values

fig = plt.figure(figsize=(100,90))

for i in features:
    plt.subplot(n_rows, n_cols, init)
    plt.xlabel(i)
    sns.countplot(data=data, x=data[i])
    init = init + 1
plt.show()

#### data is right skewed so we need to do some kind of transformation to make it bring to normal.

In [ ]:
for i in data.columns:
    print(data[i].value_counts())

# Data Preprocessing

In [ ]:
# Performing log transformation
np.seterr(divide = 'ignore')
for i in data.columns:
    #data[i] = np.where(data[i]>0, np.log1p(data[i]), 0)
    data[i] = np.log(data[i]-(min(data[i]-1)))

In [ ]:
data.head()

In [ ]:
train = data[:200000]
test = data[200000:]

In [ ]:
train = pd.concat([train,train_copy['target']], axis=1)
train.head()

In [ ]:
train.replace({'Class_1': 0, 'Class_2': 1, 'Class_3':2, 'Class_4': 3, 'Class_5': 4, 'Class_6': 5, 'Class_7': 6, 'Class_8': 7,'Class_9': 8}, inplace = True)

In [ ]:
X = train.iloc[:,0:75]
y = train.iloc[:,75:]

In [ ]:
train_tf_ds = train[:140000]
eval_tf_ds = train[140000:]

In [ ]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_tf_ds, label = 'target')

In [ ]:
eval_ds = tfdf.keras.pd_dataframe_to_tf_dataset(eval_tf_ds, label = 'target')

In [ ]:
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test, label = None)

# Model/Pipeline Building

## Tenorflow decision Forest

In [ ]:
model = tfdf.keras.GradientBoostedTreesModel(hyperparameter_template="benchmark_rank1")

In [ ]:
model.compile(metrics=["accuracy"])

In [ ]:
model.fit(train_ds)

In [ ]:
model.evaluate(eval_ds)

In [ ]:
model.summary()

In [ ]:
model.make_inspector().training_logs()

In [ ]:
logs = model.make_inspector().training_logs()

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Accuracy (out-of-bag)")

plt.subplot(1, 2, 2)
plt.plot([log.num_trees for log in logs], [log.evaluation.loss for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Logloss (out-of-bag)")

plt.show()

# Submission

In [ ]:
y_pred = model.predict(test_ds)
y_pred

In [ ]:
sample_submission.iloc[:,1:10] = y_pred
sample_submission

In [ ]:
sample_submission.to_csv('submission_df.csv',index=False)

# Please Upvote the notebook if you find it helpful.